In [ ]:
!pip install httpx==0.27.2

In [ ]:
!pip install -q openai

In [ ]:
import openai
from openai import OpenAI,OpenAIError
import pandas as pd
import os
os.environ["OPENAI_API_KEY"] = "MYAPI키입력"

In [ ]:
import openai
import pandas as pd
import os
import re

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def evaluate_sentences(messages):
    response = client.chat.completions.create(
        model="gpt모델이름",    #gpt모델이름 입력
        messages=messages,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=1,
    )
    return response.choices[0].message.content.strip()

def split_response(response):
    parts = re.split(r'\n\n|\n', response, maxsplit=1)
    if len(parts) > 1:
        return parts[0].strip(), parts[1].strip()
    else:
        return response.strip(), ""

def run_experiment(input_file, output_file):
    # 엑셀 파일에서 문장 쌍 읽기
    df = pd.read_excel(input_file)

    results = []

    for index, row in df.iterrows():
        sentence = row['sentence']

        # 공통 질문
        common_prompt = (
            "당신은 한국어를 모국어로 하는 화자이다. "
            "주어진 문장을 읽고 해당 문장의 수용성을 판단하여 답변하라."
            "여기서 수용성(acceptability)은 특정 언어 표현이나 문장이 그 언어의 화자들에 의해 자연스럽고 올바른 것으로 받아들여지는 정도를 의미한다."
                    )

        # 1단계: 더 수용성이 좋은 문장 선택 및 이유 묻기 (A/B)
        prompt_1 = [
            {"role": "system", "content": common_prompt},
            {"role": "user", "content": f"다음 문장은 수용성이 좋은 문장인가? 오직 ‘예’ 또는 ‘아니오’로만 답을 하시오.\n\n문장: {sentence}\n\n그렇게 판단한 이유는 무엇인가?"}
        ]
        response_1_full = evaluate_sentences(prompt_1)
        response_1, response_1_reason = split_response(response_1_full)

        # 2단계: 더 수용성이 좋은 문장 선택 및 이유 묻기
        prompt_2 = [
            {"role": "system", "content": common_prompt},
            {"role": "user", "content": f"다음 문장은 수용성이 좋은 문장인가? ‘예’ 또는 ‘아니오’ 중 하나만을 선택하고 주어진 문장을 그대로 반복하여 답을 하시오. \n\n문장: {sentence}\n\n그렇게 판단한 이유는 무엇인가?"}
        ]
        response_2_full = evaluate_sentences(prompt_2)
        response_2, response_2_reason = split_response(response_2_full)

        # 결과 저장
        results.append({
            "id": row['id'],
            "sentence": sentence,
            "gold_label": row['gold_label'],
            "1단계 응답": response_1,
            "1단계 이유": response_1_reason,
            "2단계 응답": response_2,
            "2단계 이유": response_2_reason,
        })

    # 결과를 데이터프레임으로 변환
    results_df = pd.DataFrame(results)

    # 결과를 엑셀 파일로 저장
    results_df.to_excel(output_file, index=False)

# 실험 실행
input_file = '입력파일.xlsx'  # 입력 엑셀 파일 경로
output_file = '출력파일.xlsx'  # 출력 엑셀 파일 경로
run_experiment(input_file, output_file)